# Exchange Rate Data

This script gathers currency exchange rate data using API calls to the AlphaVantage Monthly Foreign Exchange (FX_MONTHLY) dataset.  The data is saved in a dataframe, and is cleaned to preserve the greatest number of common datapoints across currency. The data is written to a CSV file in the Output directory.

In [1]:
# set environment
import pycountry
import requests
import json
import pandas as pd



# pull constants from config file
from config import base_country
from config import countries
from config import xchg_filename

# retrieve API key
from api_keys import akavantage


# create empty lists for our dataframe data
country_name = []
currency_name = []
currency_code = []
date = []
close_rate = []


In [2]:

# all exchange rates are in reference to the base currency
base_currency = base_country["currency_code"]

# loop through our list of currencies and pull data from AlphaVantage
for country in countries:

    # pull currency code associated with next country in list
    from_currency = countries[country]["currency_code"]
    
    # build URL for AlphaVantage
    url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={from_currency}&to_symbol={base_currency}&apikey={akavantage}"

    # request data
    try:
        response = requests.get(url).json()
    
        # loop through datapoints and populate data lists
        for record_date in response["Time Series FX (Monthly)"]:
            
            # pull date field apart
            date_list = record_date.split("-") 
            
            currency_code.append(from_currency)
            country_name.append(country)
            currency_name.append(countries[country]["currency_name"])
            date.append(f"{date_list[0]}-{date_list[1]}")
            close_rate.append(response["Time Series FX (Monthly)"][record_date]["4. close"])

    except:
        # catch error but keep going
        print(f"missing data: {from_currency}")

# retrieval takes time, so print when complete
print("Retrieval of currency exchange data complete.")
        
            


Retrieval of currency exchange data complete.


In [3]:
# build dataframe
exchange_data = pd.DataFrame({"Country": country_name, "Currency Code": currency_code,
                              "Currency": currency_name, "Date": date, "Close Rate": close_rate})
exchange_data

,Country,Currency Code,Currency,Date,Close Rate
0,European Union,EUR,Euro,2021-01,1.2077
1,European Union,EUR,Euro,2020-12,1.2214
2,European Union,EUR,Euro,2020-11,1.1925
3,European Union,EUR,Euro,2020-10,1.1646
4,European Union,EUR,Euro,2020-09,1.1722
...,...,...,...,...,...
1733,Ukraine,UAH,Ukrainian Hryvnia,2020-06,0.0373
1734,Ukraine,UAH,Ukrainian Hryvnia,2020-05,0.0370
1735,Ukraine,UAH,Ukrainian Hryvnia,2020-04,0.0369
1736,Ukraine,UAH,Ukrainian Hryvnia,2020-03,0.0361


In [4]:
# remove datapoints before 2014 and after 2020
analysis_data = exchange_data.loc[(exchange_data["Date"] >= "2014-12") & (exchange_data["Date"] <= "2019-12")]

# just a check for consistent datapoints across currencies
analysis_data.groupby("Currency Code").count()

,Country,Currency,Date,Close Rate
Currency Code,,,,
AUD,61,61,61,61
BRL,61,61,61,61
BYN,41,41,41,41
CAD,61,61,61,61
CHF,61,61,61,61
EUR,61,61,61,61
GBP,61,61,61,61
HKD,61,61,61,61
INR,61,61,61,61


In [5]:
analysis_data = analysis_data.drop(analysis_data[analysis_data["Currency Code"] == "BYN"].index)
analysis_data.groupby("Currency Code").count()

,Country,Currency,Date,Close Rate
Currency Code,,,,
AUD,61,61,61,61
BRL,61,61,61,61
CAD,61,61,61,61
CHF,61,61,61,61
EUR,61,61,61,61
GBP,61,61,61,61
HKD,61,61,61,61
INR,61,61,61,61
JPY,61,61,61,61


In [6]:
# sort dataframe by currency and year
sorted_analysis = analysis_data.sort_values(by=["Currency Code", "Date"])

# write to CSV for plotting
sorted_analysis.to_csv(xchg_filename, index=False)
sorted_analysis

,Country,Currency Code,Currency,Date,Close Rate
607,Australia,AUD,Australian Dollar,2014-12,0.8165
606,Australia,AUD,Australian Dollar,2015-01,0.7768
605,Australia,AUD,Australian Dollar,2015-02,0.7806
604,Australia,AUD,Australian Dollar,2015-03,0.7620
603,Australia,AUD,Australian Dollar,2015-04,0.7898
...,...,...,...,...,...
1669,Turkey,TRY,Turkish Lira,2019-08,0.1710
1668,Turkey,TRY,Turkish Lira,2019-09,0.1765
1667,Turkey,TRY,Turkish Lira,2019-10,0.1747
1666,Turkey,TRY,Turkish Lira,2019-11,0.1735
